In [1]:
from fedot_ind.tools.serialisation.path_lib import PROJECT_PATH
from fedot_ind.core.repository.constanst_repository import UNI_CLF_BENCH
import pandas as pd
from fedot_ind.core.repository.config_repository import DEFAULT_COMPUTE_CONFIG, DEFAULT_CLF_AUTOML_CONFIG
from fedot_ind.core.architecture.pipelines.abstract_pipeline import ApiTemplate

In [2]:
METRIC_NAMES = ('f1', 'accuracy', 'precision', 'roc_auc')
COMPUTE_CONFIG = DEFAULT_COMPUTE_CONFIG
AUTOML_LEARNING_STRATEGY = dict(timeout=2,
                                pop_size=10,
                                n_jobs=-1,
                                num_of_generations=15)

LEARNING_CONFIG = {'learning_strategy': 'from_scratch',
                   'learning_strategy_params': AUTOML_LEARNING_STRATEGY,
                   'optimisation_loss': {'quality_loss': 'accuracy'}}

INDUSTRIAL_CONFIG = {'problem': 'classification'}

API_CONFIG = {'industrial_config': INDUSTRIAL_CONFIG,
              'automl_config': DEFAULT_CLF_AUTOML_CONFIG,
              'learning_config': LEARNING_CONFIG,
              'compute_config': COMPUTE_CONFIG}
BENCHMARK_PATH = PROJECT_PATH + '/examples/real_world_examples/benchmark_example/classification/UCR_UNI_23_01_25'

In [3]:
def filter_datasets(UNI_CLF_BENCH, max_classes: int = 10, max_samples: int = 1000):
    UNI_CLF_BENCH_METADATA = pd.read_csv(PROJECT_PATH + '/fedot_ind/core/repository/data/ts_benchmark_metadata.csv')
    datasets_filtred_by_classes = UNI_CLF_BENCH_METADATA[UNI_CLF_BENCH_METADATA['Class'] <= max_classes]
    datasets_filtred_by_samples = datasets_filtred_by_classes[datasets_filtred_by_classes['Train ']
                                                              <= max_samples]
    datasets_filtred_by_samples = datasets_filtred_by_samples[datasets_filtred_by_samples['Test ']
                                                              <= max_samples]['Name'].values.tolist()
    UNI_CLF_BENCH = [x for x in UNI_CLF_BENCH if x in datasets_filtred_by_samples ]
    UNI_CLF_BENCH_METADATA = UNI_CLF_BENCH_METADATA[UNI_CLF_BENCH_METADATA['Name'].isin(datasets_filtred_by_samples)]
    return UNI_CLF_BENCH, UNI_CLF_BENCH_METADATA

In [4]:
UNI_CLF_BENCH, UNI_CLF_BENCH_METADATA = filter_datasets(UNI_CLF_BENCH)

In [5]:
api_agent = ApiTemplate(api_config=API_CONFIG, metric_list=METRIC_NAMES)

In [6]:
bench_results = api_agent.load_result(benchmark_path=BENCHMARK_PATH)

In [7]:
df_list = list(bench_results.values())

In [8]:
new_df=pd.merge(
    left=df_list[0],
    right=df_list[1],
    how='left',
    left_on=['dataset'],
    right_on=['dataset'])

In [9]:
new_df

,accuracy_pdl_rf,f1_pdl_rf,precision_pdl_rf,dataset,accuracy_rf,f1_rf,precision_rf
0,0.120,0.120,0.120,ACSF1,0.120,0.120,0.123
1,0.497,0.509,0.516,ArrowHead,0.331,0.317,0.312
2,0.167,0.145,0.132,Beef,0.133,0.124,0.117
3,0.400,0.400,0.400,BeetleFly,0.450,0.353,0.445
4,0.500,0.000,0.250,BirdChicken,0.600,0.556,0.604
5,0.280,0.285,0.299,BME,0.300,0.288,0.285
6,0.317,0.261,0.246,Car,0.267,0.274,0.318
7,0.226,0.123,0.084,CBF,0.336,0.169,0.112
8,0.254,0.158,0.294,Chinatown,0.813,0.714,0.775
9,0.429,0.429,0.431,Coffee,0.929,0.923,0.928
